# সেশন ৫ – মাল্টি-এজেন্ট অর্কেস্ট্রেটর

Foundry Local ব্যবহার করে একটি সহজ দুই-এজেন্ট পাইপলাইন (গবেষক -> সম্পাদক) প্রদর্শন করে।


### ব্যাখ্যা: নির্ভরতা ইনস্টলেশন
`foundry-local-sdk` এবং `openai` ইনস্টল করে যা স্থানীয় মডেল অ্যাক্সেস এবং চ্যাট সম্পন্ন করার জন্য প্রয়োজন। এটি ইডেমপোটেন্ট।


# পরিস্থিতি
একটি ন্যূনতম দুই-এজেন্ট অর্কেস্ট্রেটর প্যাটার্ন বাস্তবায়ন করে:
- **গবেষক এজেন্ট** সংক্ষিপ্ত তথ্যপূর্ণ পয়েন্ট সংগ্রহ করে
- **সম্পাদক এজেন্ট** নির্বাহী স্পষ্টতার জন্য পুনর্লিখন করে

প্রতিটি এজেন্টের জন্য শেয়ারড মেমোরি, মধ্যবর্তী আউটপুট ক্রমানুসারে পাস করা এবং একটি সাধারণ পাইপলাইন ফাংশন প্রদর্শন করে। এটি আরও ভূমিকা (যেমন, সমালোচক, যাচাইকর্তা) বা সমান্তরাল শাখাগুলিতে প্রসারিত করা যেতে পারে।

**পরিবেশ ভেরিয়েবলসমূহ:**
- `FOUNDRY_LOCAL_ALIAS` - ব্যবহারের জন্য ডিফল্ট মডেল (ডিফল্ট: phi-4-mini)
- `AGENT_MODEL_PRIMARY` - প্রাথমিক এজেন্ট মডেল (ALIAS কে ওভাররাইড করে)
- `AGENT_MODEL_EDITOR` - সম্পাদক এজেন্ট মডেল (ডিফল্ট প্রাথমিক মডেল)

**SDK রেফারেন্স:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local

**কিভাবে কাজ করে:**
1. **FoundryLocalManager** স্বয়ংক্রিয়ভাবে Foundry Local সার্ভিস শুরু করে
2. নির্দিষ্ট মডেল ডাউনলোড এবং লোড করে (অথবা ক্যাশড সংস্করণ ব্যবহার করে)
3. ইন্টারঅ্যাকশনের জন্য একটি OpenAI-সামঞ্জস্যপূর্ণ এন্ডপয়েন্ট প্রদান করে
4. প্রতিটি এজেন্ট বিশেষায়িত কাজের জন্য ভিন্ন মডেল ব্যবহার করতে পারে
5. বিল্ট-ইন রিট্রাই লজিক সাময়িক ব্যর্থতাগুলি দক্ষতার সাথে পরিচালনা করে

**মূল বৈশিষ্ট্যসমূহ:**
- ✅ স্বয়ংক্রিয় সার্ভিস আবিষ্কার এবং প্রারম্ভ
- ✅ মডেল লাইফসাইকেল ম্যানেজমেন্ট (ডাউনলোড, ক্যাশ, লোড)
- ✅ পরিচিত API এর জন্য OpenAI SDK সামঞ্জস্য
- ✅ এজেন্ট বিশেষায়নের জন্য মাল্টি-মডেল সাপোর্ট
- ✅ রিট্রাই লজিক সহ শক্তিশালী ত্রুটি পরিচালনা
- ✅ স্থানীয় ইনফারেন্স (ক্লাউড API প্রয়োজন নেই)


In [16]:
# Install dependencies
!pip install -q foundry-local-sdk openai

### ব্যাখ্যা: কোর ইমপোর্ট এবং টাইপিং
এজেন্ট মেসেজ সংরক্ষণে ডাটাক্লাস এবং টাইপিং হিন্টস পরিচিত করানো হয়েছে স্পষ্টতার জন্য। পরবর্তী এজেন্ট কার্যক্রমের জন্য Foundry Local ম্যানেজার এবং OpenAI ক্লায়েন্ট ইমপোর্ট করা হয়েছে।


In [17]:
from dataclasses import dataclass, field
from typing import List
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI

### ব্যাখ্যা: মডেল ইনিশিয়ালাইজেশন (SDK প্যাটার্ন)
Foundry Local Python SDK ব্যবহার করে শক্তিশালী মডেল ব্যবস্থাপনা করা হয়:
- **FoundryLocalManager(alias)** - স্বয়ংক্রিয়ভাবে সার্ভিস শুরু করে এবং alias দ্বারা মডেল লোড করে
- **get_model_info(alias)** - alias-কে নির্দিষ্ট মডেল ID-তে রূপান্তর করে
- **manager.endpoint** - OpenAI ক্লায়েন্টের জন্য সার্ভিস এন্ডপয়েন্ট প্রদান করে
- **manager.api_key** - API কী প্রদান করে (লোকাল ব্যবহারের জন্য ঐচ্ছিক)
- বিভিন্ন এজেন্টের জন্য আলাদা মডেল সমর্থন করে (প্রাইমারি বনাম এডিটর)
- স্থিতিশীলতার জন্য বিল্ট-ইন রিট্রাই লজিক এবং এক্সপোনেনশিয়াল ব্যাকঅফ
- সার্ভিস প্রস্তুত কিনা তা নিশ্চিত করতে সংযোগ যাচাই

**মূল SDK প্যাটার্ন:**
```python
manager = FoundryLocalManager(alias)
model_info = manager.get_model_info(alias)
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key)
```

**লাইফসাইকেল ব্যবস্থাপনা:**
- ম্যানেজারগুলো গ্লোবালি সংরক্ষণ করা হয় সঠিক ক্লিনআপের জন্য
- প্রতিটি এজেন্ট তাদের বিশেষায়নের জন্য আলাদা মডেল ব্যবহার করতে পারে
- স্বয়ংক্রিয় সার্ভিস ডিসকভারি এবং সংযোগ পরিচালনা
- ব্যর্থতার ক্ষেত্রে এক্সপোনেনশিয়াল ব্যাকঅফ সহ গ্রেসফুল রিট্রাই

এটি এজেন্ট অর্কেস্ট্রেশন শুরু হওয়ার আগে সঠিক ইনিশিয়ালাইজেশন নিশ্চিত করে।

**রেফারেন্স:** https://github.com/microsoft/Foundry-Local/tree/main/sdk/python/foundry_local


In [18]:
import time

# Environment configuration
PRIMARY_ALIAS = os.getenv('AGENT_MODEL_PRIMARY', os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-4-mini'))
EDITOR_ALIAS = os.getenv('AGENT_MODEL_EDITOR', PRIMARY_ALIAS)

# Store managers globally for proper lifecycle management
primary_manager = None
editor_manager = None

def init_model(alias: str, max_retries: int = 3):
    """Initialize Foundry Local manager with retry logic.
    
    Args:
        alias: Model alias to initialize
        max_retries: Number of retry attempts with exponential backoff
    
    Returns:
        Tuple of (manager, client, model_id, endpoint)
    """
    delay = 2.0
    last_err = None
    
    for attempt in range(1, max_retries + 1):
        try:
            print(f"[Init] Starting Foundry Local for '{alias}' (attempt {attempt}/{max_retries})...")
            
            # Initialize manager - this starts the service and loads the model
            manager = FoundryLocalManager(alias)
            
            # Get model info to retrieve the actual model ID
            model_info = manager.get_model_info(alias)
            model_id = model_info.id
            
            # Create OpenAI client with manager's endpoint
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key or 'not-needed'
            )
            
            # Verify the connection with a simple test
            models = client.models.list()
            print(f"[OK] Initialized '{alias}' -> {model_id} at {manager.endpoint}")
            
            return manager, client, model_id, manager.endpoint
            
        except Exception as e:
            last_err = e
            if attempt < max_retries:
                print(f"[Retry {attempt}/{max_retries}] Failed to init '{alias}': {e}")
                print(f"[Retry] Waiting {delay:.1f}s before retry...")
                time.sleep(delay)
                delay *= 2
            else:
                print(f"[ERROR] Failed to initialize '{alias}' after {max_retries} attempts")
    
    raise RuntimeError(f"Failed to initialize '{alias}' after {max_retries} attempts: {last_err}")

# Initialize primary model (for researcher)
print(f"\n{'='*80}")
print(f"Initializing Primary Model: {PRIMARY_ALIAS}")
print('='*80)
primary_manager, primary_client, PRIMARY_MODEL_ID, primary_endpoint = init_model(PRIMARY_ALIAS)

# Initialize editor model (may be same as primary)
if EDITOR_ALIAS != PRIMARY_ALIAS:
    print(f"\n{'='*80}")
    print(f"Initializing Editor Model: {EDITOR_ALIAS}")
    print('='*80)
    editor_manager, editor_client, EDITOR_MODEL_ID, editor_endpoint = init_model(EDITOR_ALIAS)
else:
    print(f"\n[Info] Editor using same model as primary")
    editor_manager = primary_manager
    editor_client, EDITOR_MODEL_ID = primary_client, PRIMARY_MODEL_ID
    editor_endpoint = primary_endpoint

print(f"\n{'='*80}")
print(f"[Configuration Summary]")
print('='*80)
print(f"  Primary Agent:")
print(f"    - Alias: {PRIMARY_ALIAS}")
print(f"    - Model: {PRIMARY_MODEL_ID}")
print(f"    - Endpoint: {primary_endpoint}")
print(f"\n  Editor Agent:")
print(f"    - Alias: {EDITOR_ALIAS}")
print(f"    - Model: {EDITOR_MODEL_ID}")
print(f"    - Endpoint: {editor_endpoint}")
print('='*80)



Initializing Primary Model: phi-4-mini
[Init] Starting Foundry Local for 'phi-4-mini' (attempt 1/3)...
[OK] Initialized 'phi-4-mini' -> Phi-4-mini-instruct-cuda-gpu:4 at http://127.0.0.1:59959/v1

Initializing Editor Model: gpt-oss-20b
[Init] Starting Foundry Local for 'gpt-oss-20b' (attempt 1/3)...
[OK] Initialized 'gpt-oss-20b' -> gpt-oss-20b-cuda-gpu:1 at http://127.0.0.1:59959/v1

[Configuration Summary]
  Primary Agent:
    - Alias: phi-4-mini
    - Model: Phi-4-mini-instruct-cuda-gpu:4
    - Endpoint: http://127.0.0.1:59959/v1

  Editor Agent:
    - Alias: gpt-oss-20b
    - Model: gpt-oss-20b-cuda-gpu:1
    - Endpoint: http://127.0.0.1:59959/v1


### ব্যাখ্যা: Agent & Memory ক্লাসসমূহ
`AgentMsg` নামে একটি হালকা ওজনের ক্লাস সংজ্ঞায়িত করা হয়েছে যা মেমোরি এন্ট্রিগুলোর জন্য ব্যবহৃত হয় এবং `Agent` ক্লাসটি নিম্নলিখিত বিষয়গুলোকে অন্তর্ভুক্ত করে:
- **System role** - এজেন্টের ব্যক্তিত্ব এবং নির্দেশনা
- **Message history** - কথোপকথনের প্রেক্ষাপট সংরক্ষণ করে
- **act() method** - সঠিক ত্রুটি পরিচালনার সাথে কার্য সম্পাদন করে

এজেন্ট বিভিন্ন মডেল (প্রাথমিক বনাম সম্পাদক) ব্যবহার করতে পারে এবং প্রতিটি এজেন্টের জন্য আলাদা প্রেক্ষাপট বজায় রাখে। এই প্যাটার্নটি নিম্নলিখিত সুবিধা প্রদান করে:
- কার্যক্রমের মধ্যে মেমোরি সংরক্ষণ
- প্রতিটি এজেন্টের জন্য নমনীয় মডেল নির্ধারণ
- ত্রুটি বিচ্ছিন্নকরণ এবং পুনরুদ্ধার
- সহজ চেইনিং এবং অর্কেস্ট্রেশন


In [19]:
@dataclass
class AgentMsg:
    role: str
    content: str

@dataclass
class Agent:
    name: str
    system: str
    client: OpenAI = None  # Allow per-agent client assignment
    model_id: str = None   # Allow per-agent model
    memory: List[AgentMsg] = field(default_factory=list)

    def _history(self):
        """Return chat history in OpenAI messages format including system + memory."""
        msgs = [{'role': 'system', 'content': self.system}]
        for m in self.memory[-6:]:  # Keep last 6 messages to avoid context overflow
            msgs.append({'role': m.role, 'content': m.content})
        return msgs

    def act(self, prompt: str, temperature: float = 0.4, max_tokens: int = 300):
        """Send a prompt, store user + assistant messages in memory, and return assistant text.
        
        Args:
            prompt: User input/task for the agent
            temperature: Sampling temperature (0.0-1.0)
            max_tokens: Maximum tokens to generate
        
        Returns:
            Assistant response text
        """
        # Use agent-specific client/model or fall back to primary
        client_to_use = self.client or primary_client
        model_to_use = self.model_id or PRIMARY_MODEL_ID
        
        self.memory.append(AgentMsg('user', prompt))
        
        try:
            # Build messages including system prompt and history
            messages = self._history() + [{'role': 'user', 'content': prompt}]
            
            resp = client_to_use.chat.completions.create(
                model=model_to_use,
                messages=messages,
                max_tokens=max_tokens,
                temperature=temperature,
            )
            
            # Validate response
            if not resp.choices:
                raise RuntimeError("No completion choices returned")
            
            out = resp.choices[0].message.content or ""
            
            if not out:
                raise RuntimeError("Empty response content")
            
        except Exception as e:
            out = f"[ERROR:{self.name}] {type(e).__name__}: {str(e)}"
            print(f"[Agent Error] {self.name}: {type(e).__name__}: {str(e)}")
        
        self.memory.append(AgentMsg('assistant', out))
        return out

print("[INFO] Agent classes initialized with Foundry SDK support")
print(f"[INFO] Using OpenAI SDK version: {OpenAI.__module__}")


[INFO] Agent classes initialized with Foundry SDK support
[INFO] Using OpenAI SDK version: openai


### ব্যাখ্যা: সমন্বিত পাইপলাইন
দুটি বিশেষায়িত এজেন্ট তৈরি করে:
- **গবেষক**: প্রাথমিক মডেল ব্যবহার করে, তথ্য সংগ্রহ করে
- **সম্পাদক**: আলাদা মডেল ব্যবহার করতে পারে (যদি কনফিগার করা হয়), তথ্য পরিমার্জন ও পুনর্লিখন করে

`pipeline` ফাংশন:
1. গবেষক কাঁচা তথ্য সংগ্রহ করে
2. সম্পাদক এক্সিকিউটিভ-প্রস্তুত আউটপুটে পরিমার্জন করে
3. মধ্যবর্তী এবং চূড়ান্ত ফলাফল উভয়ই প্রদান করে

এই প্যাটার্নটি সক্ষম করে:
- মডেলের বিশেষায়ন (ভিন্ন ভূমিকার জন্য ভিন্ন মডেল)
- বহু-পর্যায়ের প্রক্রিয়ার মাধ্যমে গুণগত মান উন্নয়ন
- তথ্য রূপান্তরের ট্রেসযোগ্যতা
- আরও এজেন্ট বা সমান্তরাল প্রক্রিয়ায় সহজ সম্প্রসারণ


In [ ]:
# Create specialized agents with optional model assignment
researcher = Agent(
    name='Researcher',
    system='You collect concise factual bullet points.',
    client=primary_client,
    model_id=PRIMARY_MODEL_ID
)

editor = Agent(
    name='Editor',
    system='You rewrite content for clarity and an executive, action-focused tone.',
    client=editor_client,
    model_id=EDITOR_MODEL_ID
)

def pipeline(q: str, verbose: bool = True):
    """Execute multi-agent pipeline: Researcher -> Editor.
    
    Args:
        q: User question/task
        verbose: Print intermediate outputs
    
    Returns:
        Dictionary with research, final outputs, and metadata
    """
    if verbose:
        print(f"[Pipeline] Question: {q}\n")
    
    # Stage 1: Research
    if verbose:
        print("[Stage 1: Research]")
    research = researcher.act(q)
    if verbose:
        print(f"Output: {research[:200]}...\n")
    
    # Stage 2: Editorial refinement
    if verbose:
        print("[Stage 2: Editorial Refinement]")
    rewrite = editor.act(
        f"Rewrite professionally with a 1-sentence executive summary first. "
        f"Improve clarity, keep bullet structure if present. Source:\n{research}"
    )
    if verbose:
        print(f"Output: {rewrite[:200]}...\n")
    
    return {
        'question': q,
        'research': research,
        'final': rewrite,
        'models': {
            'researcher': PRIMARY_MODEL_ID,
            'editor': EDITOR_MODEL_ID
        }
    }

# Execute sample pipeline
print("="*80)
result = pipeline('Explain why edge AI matters for compliance and latency.')
print("="*80)
print("\n[FINAL OUTPUT]")
print(result['final'])
print("\n[METADATA]")
print(f"Models used: {result['models']}")
result

[Pipeline] Question: Explain why edge AI matters for compliance and latency.

[Stage 1: Research]
Output: - **Data Sovereignty**: Edge AI allows data to be processed locally, which can help organizations comply with regional data protection regulations by keeping sensitive information within the borders o...

[Stage 2: Editorial Refinement]


### ব্যাখ্যা: পাইপলাইন কার্যকরকরণ ও ফলাফল
একটি কমপ্লায়েন্স + লেটেন্সি থিমযুক্ত প্রশ্নে মাল্টি-এজেন্ট পাইপলাইন কার্যকর করে দেখানো হয়েছে:
- বহু-পর্যায়ের তথ্য রূপান্তর
- এজেন্টের বিশেষীকরণ এবং সহযোগিতা
- পরিমার্জনের মাধ্যমে আউটপুটের গুণমান উন্নয়ন
- ট্রেসেবিলিটি (মধ্যবর্তী এবং চূড়ান্ত আউটপুট সংরক্ষিত)

**ফলাফলের কাঠামো:**
- `question` - মূল ব্যবহারকারীর প্রশ্ন
- `research` - কাঁচা গবেষণার আউটপুট (তথ্যভিত্তিক পয়েন্ট)
- `final` - পরিমার্জিত নির্বাহী সারাংশ
- `models` - প্রতিটি পর্যায়ে কোন মডেলগুলো ব্যবহার করা হয়েছে

**বিস্তারের ধারণা:**
1. গুণগত পর্যালোচনার জন্য একটি ক্রিটিক এজেন্ট যোগ করা
2. বিভিন্ন দিকের জন্য সমান্তরাল গবেষণা এজেন্ট বাস্তবায়ন করা
3. তথ্য যাচাইয়ের জন্য একটি ভেরিফায়ার এজেন্ট যোগ করা
4. বিভিন্ন জটিলতার স্তরের জন্য বিভিন্ন মডেল ব্যবহার করা
5. পুনরাবৃত্তি উন্নতির জন্য ফিডব্যাক লুপ বাস্তবায়ন করা


### উন্নত: কাস্টম এজেন্ট কনফিগারেশন

এজেন্টের আচরণ কাস্টমাইজ করতে চেষ্টা করুন, initialization সেল চালানোর আগে পরিবেশ ভেরিয়েবল পরিবর্তন করুন:

**উপলব্ধ মডেলসমূহ:**
- টার্মিনালে `foundry model ls` ব্যবহার করে সমস্ত উপলব্ধ মডেল দেখুন
- উদাহরণ: phi-4-mini, phi-3.5-mini, qwen2.5-7b, llama-3.2-3b, ইত্যাদি।


In [ ]:
# Example: Use different models for different agents
# Uncomment and modify as needed:

# import os
# os.environ['AGENT_MODEL_PRIMARY'] = 'phi-4-mini'      # Fast, good for research
# os.environ['AGENT_MODEL_EDITOR'] = 'qwen2.5-7b'       # Higher quality for editing

# Then restart the kernel and re-run all cells

# Test with different questions
test_questions = [
    "What are 3 key benefits of using small language models?",
    "How does RAG improve AI accuracy?",
    "Why is local inference important for privacy?"
]

print("Testing pipeline with multiple questions:\n")
for i, q in enumerate(test_questions, 1):
    print(f"\n{'='*80}")
    print(f"Question {i}: {q}")
    print('='*80)
    r = pipeline(q, verbose=False)
    print(f"\n[FINAL]: {r['final'][:300]}...")
    print(f"[Models]: Researcher={r['models']['researcher']}, Editor={r['models']['editor']}")


Testing pipeline with multiple questions:


Question 1: What are 3 key benefits of using small language models?

[FINAL]: <|channel|>analysis<|message|>The user wants a rewrite of the entire block of text. The rewrite should be professional, include a one-sentence executive summary first, improve clarity, keep bullet structure if present. The user has provided a large amount of text. The user wants a rewrite of that te...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 2: How does RAG improve AI accuracy?

[FINAL]: <|channel|>final<|message|>**RAG (Retrieval‑Augmented Generation) empowers AI to produce highly accurate, contextually relevant responses by combining a retrieval system with a large language model (LLM).**<|return|>...
[Models]: Researcher=Phi-4-mini-instruct-cuda-gpu:4, Editor=gpt-oss-20b-cuda-gpu:1

Question 3: Why is local inference important for privacy?

[FINAL]: <|channel|>final<|message|>**Local inference—processing data d


---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসাধ্য সঠিকতা নিশ্চিত করার চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। মূল ভাষায় থাকা নথিটিকে প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ সুপারিশ করা হয়। এই অনুবাদ ব্যবহারের ফলে কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যা হলে আমরা দায়বদ্ধ থাকব না।
